# Fetche EIS Fire files using Cognito

## Create AWS S3 Short Term Access Credentials

In [2]:
import boto3
from cognito_client import CognitoClient

ModuleNotFoundError: No module named 'cognito_client'

## Initiate the CognitoClient

Note: You will either need to pass the appropriate Cognito Identity Pool ID, User Pool Id, and Client ID to the instantiation of `CognitoClient()` or create an `.env` with the following content

In [2]:
client = CognitoClient()

In [3]:
_ = client.login()

········


## Use AWS short-term access credentials to call AWS via the CLI

You can now use the AWS CLI with credentials generated by the following code:

In [5]:
 creds = client.get_aws_credentials()

## Use boto3 for AWS operations

You can use boto3 for AWS operations. At this time the roles associated with Cognito identities are associated with S3 access policies, so you will want to refer to the [S3 Boto3 Docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html).

Below is an example for how to put and remove a test file.

In [6]:
session = boto3.Session(aws_access_key_id=creds["AccessKeyId"], 
                        aws_secret_access_key=creds["SecretKey"],
                        aws_session_token=creds["SessionToken"])

client = session.client('s3')

In [7]:
import os
years = ['2019', '2020']
bucket = 'veda-data-store-staging'
download_data_dir = 'data/eis_fire'

for year in years:
    prefix = f'EIS/other/feds-WesternUS/{year}/Largefire/00-latest'
    year_dir = f"{download_data_dir}/{year}"
    if not os.path.exists(year_dir):
        os.makedirs(year_dir)    
    response = client.list_objects_v2(
        Bucket=bucket,
        Prefix=prefix)
    response_status = response['ResponseMetadata']['HTTPStatusCode']
    if response_status == 200:
        print(f"Got a {response['ResponseMetadata']['HTTPStatusCode']} status code response.")
    else:
        print(f"Got unexpected response: {response}")  
    objects_to_download = [{'key': r['Key'], 'size': r['Size']} for r in response['Contents']]
    print(objects_to_download)
    for obj in objects_to_download:
        object_key, object_size = obj.values()
        if object_size >= 1e+9:
            print(f'{object_key} is {object_size}. Do you really want to download a gb of data?')
        else:
            filename = object_key.split('/')[-1]
            download_location = f"{year_dir}/{filename}"
            print(f"Downloading {filename} to {download_location}")
            client.download_file(bucket, object_key, download_location)    

Got a 200 status code response.
[{'key': 'EIS/other/feds-WesternUS/2019/Largefire/00-latest/fireline.fgb', 'size': 43995344}, {'key': 'EIS/other/feds-WesternUS/2019/Largefire/00-latest/newfirepix.fgb', 'size': 5531400}, {'key': 'EIS/other/feds-WesternUS/2019/Largefire/00-latest/perimeter.fgb', 'size': 254353056}]
Got a 200 status code response.
[{'key': 'EIS/other/feds-WesternUS/2020/Largefire/00-latest/fireline.fgb', 'size': 36942848}, {'key': 'EIS/other/feds-WesternUS/2020/Largefire/00-latest/newfirepix.fgb', 'size': 12485496}, {'key': 'EIS/other/feds-WesternUS/2020/Largefire/00-latest/perimeter.fgb', 'size': 196043928}]
